In [12]:
import math
import networkx as nx
import wikipediaapi, time, json,requests,os
from collections import Counter

In [13]:
def load_people_json(filename):
    with open(filename, "r") as pfile: 
        return json.load(pfile)

filename="politicians.json"
#if not os.path.isfile(filename):
#    politicians=filter_persons_by(occupation="politician")
#    fill_in_genders(politicians)
#    save_people_json(politicians,filename)
    
politicians=load_people_json(filename)

## The code below fills in summaries, language editions and links from wikipedia.
## The fetching takes place in batches of 1000 queries after which the data is saved to disk.
#while fetch_summaries(politicians,batch_size=1000): save_people_json(politicians,filename)
#while fetch_langs(politicians,batch_size=1000): save_people_json(politicians,filename)
#while fetch_links(politicians,batch_size=1000): save_people_json(politicians,filename)
#save_people_json(politicians,filename)


In [14]:
politicians['Benedict_Calvert,_4th_Baron_Baltimore']

{'#DBpURL': 'http://dbpedia.org/resource/Benedict_Calvert,_4th_Baron_Baltimore',
 'ID': '21',
 'WikiURL': 'http://en.wikipedia.org/wiki/Benedict_Calvert,_4th_Baron_Baltimore',
 'gender': 'M',
 'name': "[' (the right honourable) ', ' the lord baltimore ']",
 'birthDate': "[' 1679-03-21 ']",
 'deathDate': "[' 1715-04-16 ']",
 'occupation': "[' politician ']",
 'nationality': 'NA',
 'party': 'NA',
 'summary': "Benedict Leonard Calvert, 4th Baron Baltimore (21 March 1679 – 16 April 1715) was an English nobleman and politician. He was the second son of Charles Calvert, 3rd Baron Baltimore (1637–1715) by Jane Lowe, and became his father's heir upon the death of his elder brother Cecil in 1681. The 3rd Lord Baltimore was a devout Roman Catholic, and had lost his title to the Province of Maryland shortly after the events of the Glorious Revolution in 1688, when the Protestant monarchs William III and Mary II acceded to the British throne. Benedict Calvert made strenuous attempts to have his fa

# Task B: Initial data analysis (100 points)

Construct network: how many links are there

Question 1 (20 points)

Your first task is to construct a network of links between wikipages of the politicians. Use Graph object in NetworkX to construct an undirected network. There should be an undirected edge between nodes A and B if either there is a link from page of A to B or link from B to A, or both. Note that if you just add edges to the Graph object in Networkx, this should be automatically taken care of. Further, do not add self-edge, i.e., check that you don’t add edge from a politician to themself. You are now ready to start exploring the structure of the network! How many edges does your network have?

Use the next cell to inspect how the data looks like for a single politician. You might want to go through a bit more data manually to see how it looks like. For example, it will be useful to know how the genders are marked in the data.

In [15]:
network = nx.Graph()
network.add_nodes_from(list(politicians.keys()))

politicians_in_graph = {}
for name, data in politicians.items():
    # The links attribute is of type List[str]
    for link in data.get("links", []):
        if name != link:
            network.add_edge(name, link)
            politicians_in_graph[name] = data
num_edges = network.number_of_edges()
print("Number of edges in the network:", num_edges)

Number of edges in the network: 10294


Network degrees by gender

Question 2 (20 points)

Next, compute the average degree of males and females in the network. Be sure to include nodes with degree 0, i.e., nodes that don’t have any links. What do you observe? This is multiple choice

1. Females have higher degree than males, with average degree around 7.2 as compared to male average degree around 4.1 (correct)
2. Females have higher degree than males, with average degree around 5.3 as compared to male average degree around 4.5.
3. Males have higher degree than females, with average degree around 7.2 as compared to female average degree around 4.1.
4. Males have higher degree than females, with average degree around 5.3 as compared to female average degree around 4.5.

In [16]:
# Create dictionaries to store the degrees of males and females
male_degrees = {}
female_degrees = {}

# Loop over all the nodes in the network
for node in network.nodes():
    # Get the gender of the node
    gender = politicians[node]["gender"]
    # Get the degree of the node
    degree = network.degree(node)
    # Add the degree to the appropriate dictionary
    if gender == "M":
        male_degrees[node] = degree
    elif gender == "F":
        female_degrees[node] = degree

# Calculate the average degree of males and females
male_degree_avg = sum(male_degrees.values()) / len(male_degrees)
female_degree_avg = sum(female_degrees.values()) / len(female_degrees)

print("Average degree of males:", male_degree_avg)
print("Average degree of females:", female_degree_avg)


Average degree of males: 4.087830687830688
Average degree of females: 7.176165803108808


Summaries by gender: bag-of-words

Question 3 (20 points)

Your next task is to construct bag-of-words for all the summary texts of all politicians, male politicians, and female politicians. That is, you count the times each word appears in summaries of these three categories. Sort the list by frequency and inspect the most frequent ones. In all three sets of politicians (all, male, female), the three most common words should be the stopwords 1. “the”, 2. “of”, 3. “and”. What is the 4th most common word both in male and female summaries?

In [17]:
# Concatenate all summaries into a single string
all_summaries = " ".join([data["summary"] for data in politicians.values()])

# Separate summaries by gender
male_summaries = " ".join([data["summary"] for data in politicians.values() if data["gender"] == "M"])
female_summaries = " ".join([data["summary"] for data in politicians.values() if data["gender"] == "F"])

# Count occurrences of each word
all_word_counts = Counter(all_summaries.lower().split())
male_word_counts = Counter(male_summaries.lower().split())
female_word_counts = Counter(female_summaries.lower().split())

# Sort words by frequency
all_word_freq = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)
male_word_freq = sorted(male_word_counts.items(), key=lambda x: x[1], reverse=True)
female_word_freq = sorted(female_word_counts.items(), key=lambda x: x[1], reverse=True)

print("4th most common word in both male and female summaries:")
print(male_word_freq[3])
print(female_word_freq[3])


4th most common word in both male and female summaries:
('in', 8901)
('in', 1082)


Summaries by gender: tf-idf

Question 4 (40 points)

Your final task is to compute the average tf-idf scores for words in each category. To do this, first compute the average tf score by simply dividing the bag-of-words number you computed in the previous question by the total number of politicians of a certain kind. For example, if there are 386 female politicians, and the word “the” appears 2293 times in their summaries, the tf value for “the for females is 2293/386. The idf score you count by first computing the document frequency df, which is the number of summary texts wherein the word appears at least once and divide that by the total number of politicians. For example, if the word “the” appears in 6149 of all 6880 summary texts then the df value is 6149/6880. The inverse document frequency score, i.e., the idf score, is then the logarithm of the inverse of the df, for example for the word “the” idf = log(6880/6149). Use the natural logarithm here. The tf-idf score is thus the multiplication of these two scores, that is, tf*idf, and its value for females and the word “the” is 2293/386 * log(6880/6149). Inspect the top tf-idf scores for males and females. For females the top score is given by “she”. What is the word with second highest score for females?

In [18]:
print(all_word_freq)

[('the', 36237), ('of', 27826), ('and', 16770), ('in', 14832), ('a', 14579), ('was', 11655), ('he', 11570), ('as', 9606), ('to', 8568), ('from', 6625), ('is', 5625), ('for', 4734), ('served', 4274), ('politician', 3989), ('==', 3916), ('his', 3769), ('mexican', 3533), ('(born', 3010), ('member', 2981), ('on', 2892), ('–', 2661), ('an', 2593), ('with', 2304), ('party', 2267), ('deputy', 2117), ('she', 2110), ('congress', 2052), ('minister', 1964), ('references', 1949), ('representing', 1863), ('at', 1695), ('party.', 1676), ('legislature', 1648), ('national', 1643), ('by', 1629), ('who', 1519), ('also', 1517), ('state', 1373), ('after', 1276), ('until', 1253), ('may', 1216), ('politician.', 1159), ('president', 1152), ('elected', 1152), ('june', 1149), ('first', 1123), ('january', 1070), ('july', 1035), ('march', 1018), ('during', 1007), ('september', 997), ('that', 996), ('november', 989), ('december', 983), ('became', 982), ('october', 980), ('has', 979), ('august', 966), ('de', 953),

In [19]:
female_politicians = {name: data for name, data in politicians.items() if data["gender"] == "F"}

num_female = len(female_politicians)
num_politicians = len(politicians)

# Compute tf-idf scores for each word
female_tf_idf = {}

female_tf = {}

female_idf = {}

for word, count in female_word_freq[0:100]:
    # Compute term frequency for females
    female_tf[word] = count / num_female
    # Compute document frequency for females
    df_female = len([name for name, data in politicians.items() if word in data["summary"].lower()]) 
    # Compute idf for females
    female_idf[word] = math.log(num_politicians / df_female)
    # Compute tf-idf for females
    female_tf_idf[word] = female_tf[word] * female_idf[word]

# Sort words by tf-idf score for females
female_word_scores = sorted(female_tf_idf.items(), key=lambda x: x[1], reverse=True)

# Print top 10 words and their tf-idf scores for females
for i, (word, score) in enumerate(female_word_scores[:10]):
    print(f"{i+1}. {word}: {score}")

# Answer to question
print("Word with second highest tf-idf score for females:", female_word_scores[1][0])


1. she: 4.012691205987612
2. her: 1.2063609197124032
3. for: 0.9718456524439745
4. and: 0.841908624224511
5. member: 0.8364786053682933
6. was: 0.8232837781949914
7. minister: 0.7118654775698704
8. elected: 0.6990721896763458
9. from: 0.682286513622365
10. former: 0.676511960668353
Word with second highest tf-idf score for females: her
